In [1]:
import sys
sys.path.append('..')

import numpy as np
from corpus_preprocessing import utils
from character_choice import *

In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [3]:
scenes = utils.load_scenes_from_folder('../raws')

In [4]:
filter_stage_directions(scenes)
scenes[0]

[Scene: Central Perk, Chandler, Joey, Phoebe, and Monica are there.]
monica: There's nothing to tell! He's just some guy I work with!
joey: C'mon, you're going out with the guy! There's gotta be something wrong with him!
chandler: All right Joey, be nice. So does he have a hump? A hump and a hairpiece?
phoebe: Wait, does he eat chalk?
phoebe: Just, 'cause, I don't want her to go through what I went through with Carl- oh!
monica: Okay, everybody relax. This is not even a date. It's just two people going out to dinner and- not having sex.
chandler: Sounds like a date to me.

In [5]:
character_to_id = generate_character_to_id_dict(scenes, 20)
character_to_id 

{'UNK_CHARACTER': 0,
 'END_CHARACTER': 1,
 'rachel': 2,
 'ross': 3,
 'monica': 4,
 'chandler': 5,
 'joey': 6,
 'phoebe': 7,
 'mike': 8,
 'all': 9,
 'richard': 10,
 'janice': 11,
 'mr. geller': 12,
 'charlie': 13,
 'carol': 14,
 'emily': 15,
 'mrs. geller': 16,
 'frank': 17,
 'tag': 18,
 'paul': 19,
 'gunther': 20,
 'david': 21}

In [6]:
character_sequences = scenes_to_character_sequences(scenes, character_to_id)
character_sequences[0]

[4, 6, 5, 7, 7, 4, 5]

In [7]:
len(character_sequences)

3578

In [8]:
lm = MarkovCharacterChoice(character_sequences, 3, character_to_id)
lm.get_possible_next_characters([3])

Counter({6: 0.055900621118012424,
         2: 0.15093167701863355,
         0: 0.4745341614906832,
         3: 0.031677018633540374,
         14: 0.017391304347826087,
         1: 0.07763975155279502,
         9: 0.002484472049689441,
         5: 0.07080745341614907,
         4: 0.046583850931677016,
         16: 0.0018633540372670807,
         7: 0.0453416149068323,
         10: 0.0006211180124223603,
         20: 0.004347826086956522,
         15: 0.006211180124223602,
         12: 0.003105590062111801,
         19: 0.0012422360248447205,
         18: 0.0006211180124223603,
         8: 0.0018633540372670807,
         13: 0.0068322981366459624})

In [9]:
from sklearn.model_selection import train_test_split


train, test = train_test_split(character_sequences, test_size=0.3, random_state=42)

for n in (1, 2, 3):
    lm = MarkovCharacterChoice(train, n, character_to_id)
    ppx = perplexity(lm, test, character_to_id)
    print("N = %i, Perplexity = %.5f" % (n, ppx))


N = 1, Perplexity = 10.91214
N = 2, Perplexity = 11.13090
N = 3, Perplexity = 32.90798


In [10]:
prefix = [3]

for i in range(100):
    prefix += [get_next_character(lm, prefix)]
    if prefix[-1] == character_to_id[END_CHARACTER] or len(lm.get_possible_next_characters(prefix)) == 0:
        break
        
print(prefix)

[3, 7, 3, 7, 9, 2, 7, 2, 7, 5, 3, 5, 0, 5, 4, 5, 4, 6, 2, 6, 3, 5, 3, 6, 5, 3, 5, 3, 2, 3, 4, 3, 7, 2, 7, 5, 7, 4, 7, 4, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 2, 5, 2, 7, 2, 7, 2, 4, 3, 7, 3, 7, 3, 7, 4, 5, 4, 5, 4, 11, 5, 0, 3, 1]
